In [2]:
import pandas as pd
import os
import json
# Filter for LLM spans
from collections import defaultdict
import shutil

# language_model_id = "gpt-4o-mini_baseline"
# language_model_id = "open-mistral-nemo_dspy-baseline"

In [3]:
def crop_from_results(error_txt, crop_until_results: bool = False):
  split=error_txt.split("\n")
  idx = 0
  has_found_results = False
  end_idx = len(split)-1
  for i, line in enumerate(split):
    if "[INFO] Results:" in line and not has_found_results:
        idx= i
        has_found_results = True
    if "Tests run:" in line and crop_until_results:
        end_idx = i

  return "\n".join(split[idx:end_idx])



In [4]:



# Filter for LLM spans
llm_df = grouped_df[grouped_df["openinference.span.kind"] == "LLM"]

# Group data by trace_id
trace_groups = defaultdict(list)

for _, row in llm_df.iterrows():
    trace_id = row["trace_id"]
    commit_hash = row["commit_hash"]
    language_model = row["language_model"]

    trace_groups[trace_id].append(
        {
            "commit_hash": commit_hash,
            "language_model": language_model,
            "data": row.where(pd.notna(row), None).to_dict(),
        }
    )

# Write grouped data to files
for trace_id, group_data in trace_groups.items():
    if not group_data:
        continue

    commit_hash = group_data[0]["commit_hash"]
    language_model = group_data[0]["language_model"]

    # Create directory if it doesn't exist
    directory = f"dataset/{commit_hash}/out/llm/{language_model}/"
    os.makedirs(directory, exist_ok=True)

    # Create filename using trace_id
    filename = f"{directory}{trace_id}.json"

    # Write to JSON file
    with open(filename, "w") as f:
        json.dump(group_data, f, default=str, indent=2)

print("Files have been written successfully.")

NameError: name 'grouped_df' is not defined

In [ ]:
import json
from pathlib import Path

from masterthesis.evaluation.output_success_criterion import output_success_criterion
import os


# find dataset/*/out/{language_model_id}-*-execution-errors.json 
data_path = Path(os.path.abspath(""))/"dataset"

execution_details = {}


total_error_count = 0
total_diff_error_count = 0

total_full_successes = 0
total_compiled = 0


total_attempts_compiled_count = 0
total_attempts_not_compiled_count = 0

trial_data = defaultdict(dict)


language_model_id = "claude-3-haiku@20240307"


for path in data_path.rglob(f"out/full-{language_model_id}*-execution-error*.json"):
    # I want the commit hash and the number between language model and execution-errors
    commit_hash = path.parts[-3]
    # experiment_number = path.parts[-1].split("-execution-errors.json")[0]
    # experiment_number = experiment_number.split("-")[-1]
    if "execution-errors.json" in path.parts[-1]:
        experiment_number = "trial-0"
    elif "execution-errors-1.json" in path.parts[-1]:
        experiment_number = "trial-1"
    elif "execution-errors-2.json" in path.parts[-1]:
        experiment_number = "trial-2"

    def add_to_trial_data(key:str, val: int):
        if key not in trial_data[experiment_number]:
            if isinstance(val, int):
                trial_data[experiment_number][key] = 0
            if isinstance(val, str):
                trial_data[experiment_number][key] = []

        if isinstance(val, int):
            trial_data[experiment_number][key] += val
        if isinstance(val, str):
            trial_data[experiment_number][key].append(val)

    # print(commit_hash, experiment_number)
    with open(path, "r") as f:
        execution_errors = json.load(f)
        execution_errors = execution_errors.get(commit_hash, execution_errors)
        if len(execution_errors.keys()) == 0:
            continue
        if commit_hash not in execution_details:
            execution_details[commit_hash] = []
        execution_details[commit_hash].append(execution_errors)

        # print(execution_errors.keys())

        # print(execution_errors)

        if "error" in execution_errors and execution_errors['error'] is not None:
            # print("plus one total errors")
            total_error_count += 1
            add_to_trial_data("error_count", 1)

            if not "hops" in trial_data[experiment_number]:
                trial_data[experiment_number]["hops"] = {}
            trial_data[experiment_number]["hops"][commit_hash] = len(execution_errors.get("diff_attempts", []))

            add_to_trial_data("hop", len(execution_errors.get("compiled_protocol", [])))

            if "Invalid diff: [Errno 2] No such file or directory" in execution_errors['error']:
                add_to_trial_data("error_details_hallucinated_diff_path", 1)

            if "Unfixable JVM Error" in execution_errors['error']:
                add_to_trial_data("error_details_unfixable_jvm_error", 1)
            
            if "Total tokens exceed the threshold" in execution_errors['error']:
                add_to_trial_data("error_details_total_tokens_exceed_threshold", 1)

            if "UnifiedDiffNoMatch: hunk failed to apply" in execution_errors['error']:
                add_to_trial_data("error_details_unified_diff_no_match", 1)

            if "No diff fences found in content." in execution_errors['error']:
                add_to_trial_data("error_details_no_diff_fences", 1)

            if "UnifiedDiffNoEdits: no applicable hunks found" in execution_errors['error']:
                add_to_trial_data("error_details_unified_diff_no_edits", 1)

                
        # total_diff_error_count += 1 if error_info is not None and "Invalid diff" in error_info else 0
        if "diff_attempts" in execution_errors:
            total_diff_error_count += len(execution_errors["diff_attempts"])
            add_to_trial_data("diff_error_count", total_diff_error_count)
        else:
            print("no diff attempts", commit_hash)
            print(commit_hash, execution_errors)

        if output_success_criterion(execution_errors.get("output", None)) == True:
            total_full_successes += 1
            add_to_trial_data("full_successes", 1)
            add_to_trial_data(f"full_successes-hash", commit_hash)

        if "compiled_protocol" in execution_errors and execution_errors["compiled_protocol"] is not None:
            for status in execution_errors["compiled_protocol"]:
                if status == True:
                    total_attempts_compiled_count += 1
                    add_to_trial_data("attempts_compiled", 1)
                else:
                    total_attempts_not_compiled_count += 1
                    add_to_trial_data("attempts_not_compiled", 1)
        else:
            add_to_trial_data("no_compiled_protocol", 1)
        
        if "compiled" in execution_errors:
            if execution_errors["compiled"] == True:
                total_compiled += 1
                add_to_trial_data("compiled", 1)
                add_to_trial_data(f"compiled-hash", commit_hash)

                # repro_path = Path(os.path.abspath(""))/f"dataset/{commit_hash}/out/reproduction"
                # post_path = repro_path / "post.txt"
                # pre_path = repro_path / "pre.txt"

                # if repro_path.exists() and pre_path.exists() and post_path.exists():

                #     # dataset/0305beafdecb0b28f7c94264ed20cdc4e41ff067/out/reproduction/post.txt
                #     with open(f"dataset/{commit_hash}/out/reproduction/post.txt", "r") as f:
                #         post_txt = crop_from_results(f.read(), False)

                #     with open(f"dataset/{commit_hash}/out/reproduction/pre.txt", "r") as f:
                #         pre_txt = crop_from_results(f.read(), False)

                #     if "[INFO] BUILD FAILURE" in pre_txt:
                #         add_to_trial_data("tests_broken_in_environment", 1)
                #     # print("equal", pre_txt == post_txt)
                #     # if pre_txt == post_txt:
                #     #     add_to_trial_data("tests_broken_in_environment", 1)
                #     # else:
                #     #     print("not equal")
                #     #     print(pre_txt)
                #     #     print()
                #     #     print(post_txt)
                # else:
                #     add_to_trial_data("no_repro", 1)
                    


print("total_error_count", total_error_count)
print("total_diff_error_count", total_diff_error_count)
print("total_full_successes", total_full_successes)
print("total_compiled", total_compiled)
print("total_attempts_compiled_count", total_attempts_compiled_count)
print("total_attempts_not_compiled_count", total_attempts_not_compiled_count)

print(json.dumps(trial_data, indent=4))

with open(language_model_id+"_data.json", "w") as f:
    json.dump(trial_data, f, indent=4)



In [ ]:
import json
from pathlib import Path
import os

def find_and_delete_files_with_errors():
    data_path = Path(os.path.abspath("")) / "dataset"
    error_messages = ["403 This API method requires billing", "Quota exceeded"]
    commit_hashes_with_errors = set()

    for path in data_path.rglob("out/*-execution-error*"):
        commit_hash = path.parts[-3]
        with open(path, "r") as f:
            execution_errors = json.load(f)
            for key, value in execution_errors.items():
                if isinstance(value, dict):
                    for sub_key, sub_value in value.items():
                        if any(error in str(sub_value) for error in error_messages):
                            commit_hashes_with_errors.add(commit_hash)
                            os.remove(path)
                            break
                else:
                    if any(error in str(value) for error in error_messages):
                        commit_hashes_with_errors.add(commit_hash)
                        os.remove(path)
                        break

    return commit_hashes_with_errors

commit_hashes = find_and_delete_files_with_errors()
print(commit_hashes)

# Summary Table LaTex

In [10]:
def raw_model_mapper(language_model_id):
    language_model_id = language_model_id.replace("-execution-errors.json", "")
    language_model_id = language_model_id.replace(".json", "")
    language_model_id = language_model_id.replace("meta-llama_Meta-", "")
    language_model_id = language_model_id.replace("-Instruct-Turbo", "")
    language_model_id = language_model_id.replace("open-mistral", "mistral")
    language_model_id = language_model_id.replace("-001", "")
    language_model_id = language_model_id.replace("text-bison@002", "text-bison-002")
    language_model_id = language_model_id.replace("@20240307", "")
    language_model_id = language_model_id.replace("@20240620", "")
    language_model_id = language_model_id.replace("claude-3-5-sonnet", "claude-3.5-sonnet")
    return language_model_id

In [13]:
import json
from pathlib import Path
import os
from collections import defaultdict
import pandas as pd
import re

include_full = False

def get_language_models():
    data_path = Path(os.path.abspath("")) / "dataset"
    language_models = set()
    if include_full:
        for path in data_path.rglob("out/*-execution-error*"):
            language_models.add(path.parts[-1])
    else:
        for path in data_path.rglob("out/dspy-baseline-*-execution-errors.json"):
            language_models.add(path.parts[-1])

    # print(language_models)

    return list(language_models)



def process_language_model(language_model_id):
    data_path = Path(os.path.abspath("")) / "dataset"
    execution_details = {}
    total_error_count = 0
    total_compiled = 0
    total_full_successes = 0
    total_attempts = 0

    for path in data_path.rglob(f"out/{language_model_id}*"):
        commit_hash = path.parts[-3]
        total_attempts += 1

        pretty_language_model_id = raw_model_mapper(language_model_id)
        if include_full:
            if "dspy-baseline-claude-3-haiku" == pretty_language_model_id:
                pretty_language_model_id = "claude-3-haiku-baseline"
            elif "full" in pretty_language_model_id:
                pretty_language_model_id = pretty_language_model_id.replace("full-", "")
                if not re.search(r"-execution-errors-(\d+)", pretty_language_model_id):
                    pretty_language_model_id += "-trial-1"
                else:
                    addition= 1
                    if "supplement-" in pretty_language_model_id:
                        addition = 0
                    pretty_language_model_id = re.sub(r"-execution-errors-(\d+)", lambda m: f"-trial-{int(m.group(1)) + addition}", pretty_language_model_id)
                pretty_language_model_id = pretty_language_model_id.replace("supplement-", "")
            else:
                return None
        
        if "dspy-baseline-" in pretty_language_model_id and not include_full:
            pretty_language_model_id = pretty_language_model_id.replace("dspy-baseline-", "")

        with open(path, "r") as f:
            execution_errors = json.load(f)
            execution_errors = execution_errors.get(commit_hash, execution_errors)
            
            if len(execution_errors.keys()) == 0:
                continue
            
            if commit_hash not in execution_details:
                execution_details[commit_hash] = []
            execution_details[commit_hash].append(execution_errors)

            full_success = "tests" in execution_errors and execution_errors['tests'] == True

            

            if "compiled" in execution_errors and execution_errors["compiled"] == True and not full_success:
                total_compiled += 1

            elif full_success:
                total_full_successes += 1

            elif "error" in execution_errors and execution_errors['error'] is not None:
                total_error_count += 1
    
    # if total_attempts != 65:
    #     return None
    return {
        "LM": pretty_language_model_id,
        "Attempts": total_attempts,
        "Test Success": total_full_successes,
        "Test Errors": total_compiled,
        "Other Errors": total_error_count,
    }

def split_header_for_latex(header_name: str): 
    split = header_name.split(" ")
    if len(split) == 1:
        return header_name
    return '\\begin{tabular}[c]{@{}l@{}}' + '\\\\'.join(split) + '\\end{tabular}'

def format_percentage(value, total):
    if total == 0:
        return f"{value} (0.0%)"
    percentage = (value / total) * 100
    return f"{value} ({percentage:.1f}\\%)"


language_models = get_language_models()
data = [process_language_model(lm) for lm in language_models if "bison" not in lm]
data = [d for d in data if d is not None]

assert len(data) > 0, "No data found"

df = pd.DataFrame(data)
if not include_full:
    df = df.sort_values(['Test Success', "LM"], ascending=[False,True])
else:
    df = df.sort_values(['LM'], ascending=[True])




regular_attempts = df['Attempts'].max()



success_df = df.copy()
# df['Errors'] = df.apply(lambda row: format_percentage(row['Errors'], row['Attempts']), axis=1)
if include_full:
    df = df.drop('Attempts', axis=1)
    df = df.rename(columns={"LM": "LM Trials"})
latex_columns = [split_header_for_latex(col) for col in df.columns]
df.columns = latex_columns


   
latex_table = df.to_latex(index=False, 
                            escape=False, 
                            column_format='lllrr',
                            caption=f"Few Shot Prompting with different language models (n={regular_attempts})",
                            position="H",
                            label="tab:few-shot-prompting")
    
    
print(latex_table)

\begin{table}[H]
\caption{Few Shot Prompting with different language models (n=65)}
\label{tab:few-shot-prompting}
\begin{tabular}{lllrr}
\toprule
LM & Attempts & \begin{tabular}[c]{@{}l@{}}Test\\Success\end{tabular} & \begin{tabular}[c]{@{}l@{}}Test\\Errors\end{tabular} & \begin{tabular}[c]{@{}l@{}}Other\\Errors\end{tabular} \\
\midrule
claude-3.5-sonnet & 65 & 12 & 28 & 25 \\
gemini-1.5-pro & 65 & 10 & 24 & 31 \\
gpt-4o & 65 & 8 & 26 & 30 \\
claude-3-haiku & 65 & 7 & 22 & 36 \\
Llama-3.1-70B & 65 & 4 & 7 & 54 \\
gpt-4o-mini & 65 & 4 & 11 & 50 \\
mistral-nemo & 65 & 3 & 14 & 48 \\
\bottomrule
\end{tabular}
\end{table}



# Total Time Spent Trace

In [8]:
import ijson
import json
from datetime import datetime, timezone
from pathlib import Path
import os

from tqdm import tqdm
from glob import glob
from collections import defaultdict
import re

token_pattern = r"Usage\(input_tokens=(\d+), output_tokens=(\d+)\)"

# usage=UsageInfo(prompt_tokens=2140, total_tokens=2294, completion_tokens=154)
mistral_pattern = r"UsageInfo\(prompt_tokens=(\d+), total_tokens=(\d+), completion_tokens=(\d+)\)"

# Initializing defaultdict for token sums
token_sums = defaultdict(lambda: {'input_tokens': 0, 'output_tokens': 0})

def process_large_jsonl(file_path):
    min_start_time = None
    max_end_time = None


    # {    "name": "log_usage",    "context": {        "trace_id": "0x38aa026246e37c16aefb2492f7c60b07",        "span_id": "0xccacef7a95f9840e",        "trace_state": "[]"    },    "kind": "SpanKind.INTERNAL",    "parent_id": "0xc831f4697ab76a71",    "start_time": "2024-08-12T13:04:51.993642Z",    "end_time": "2024-08-12T13:04:51.993748Z",    "status": {        "status_code": "UNSET"    },    "attributes": {        "usage": "{\"prompt_tokens\": 12495, \"total_tokens\": 12498, \"completion_tokens\": 3}"    },    "events": [],    "links": [],    "resource": {        "attributes": {},        "schema_url": ""    }}

    with open(file_path, 'rb') as file:
        parser = ijson.parse(file, multiple_values=True)
        current_model = None  # To track the current model name
        in_model_request = False  # Track if we are inside a model request

        for prefix, event, value in parser:
            # Handle start_time and end_time globally (independent of model requests)
            if prefix.endswith('start_time') and event == 'string':
                current_start = datetime.fromisoformat(value.rstrip('Z')).replace(tzinfo=timezone.utc)
                if min_start_time is None or current_start < min_start_time:
                    min_start_time = current_start
            elif prefix.endswith('end_time') and event == 'string':
                current_end = datetime.fromisoformat(value.rstrip('Z')).replace(tzinfo=timezone.utc)
                if max_end_time is None or current_end > max_end_time:
                    max_end_time = current_end

            # Detect the model name and track if we're inside a model request
            if prefix.endswith('llm.model_name') and event == 'string':
                current_model = value
                in_model_request = True  # Enter model request
            elif prefix.endswith('name') and event == 'string':
                in_model_request = False  # Exit model request if name field changes

            # Process only if inside a model request
            if in_model_request and current_model:
                # Special handling for Claude models using regex
                if current_model.startswith('claude'):
                    if prefix.endswith('output.value') and event == 'string':
                        match = re.search(token_pattern, value)
                        if match:
                            token_sums[current_model]['input_tokens'] += int(match.group(1))
                            token_sums[current_model]['output_tokens'] += int(match.group(2))
                if current_model.startswith('open-mistral'):
                    if prefix.endswith('output.value') and event == 'string':
                        match = re.search(mistral_pattern, value)
                        if match:
                            token_sums[current_model]['input_tokens'] += int(match.group(1))
                            token_sums[current_model]['output_tokens'] += int(match.group(3))
                if current_model.startswith('gpt') or current_model.startswith('meta_llama'): 
                    if prefix.endswith('output.value') and event == 'string':
                        output_data = json.loads(value)
                        usage_data = output_data.get('usage', {"usage": {"prompt_tokens": 0, "completion_tokens": 0}})
                        token_sums[current_model]['input_tokens'] += usage_data.get('prompt_tokens', 0)
                        token_sums[current_model]['output_tokens'] += usage_data.get('completion_tokens', 0)
                    

            # Handle non-Claude models, assuming attributes.usage contains token data
            if prefix.endswith('attributes.usage') and event == 'string':
                usage_data = json.loads(value)
                token_sums[current_model]['input_tokens'] += usage_data.get('prompt_tokens', 0)
                token_sums[current_model]['output_tokens'] += usage_data.get('completion_tokens', 0)

    if min_start_time and max_end_time:
        duration = max_end_time - min_start_time
        return min_start_time, max_end_time, duration
    else:
        return None, None, None


# glob *_augmented_prompt.json  
glob_paths = glob(os.path.join((os.path.abspath("")), "trace_dspy-baseline*.json"))

# glob_paths = glob(os.path.join((os.path.abspath("")), "trace_dspy-baseline_meta*.json"))

duration_by_language_model = defaultdict(dict)

for file_path in tqdm(glob_paths):
    language_model_id = Path(file_path).parts[-1].replace("trace_dspy-baseline_", "").replace(".json", "")

    # Usage


    min_start, max_end, total_duration = process_large_jsonl(file_path)

    duration_by_language_model[language_model_id] = {"min_start": min_start, "max_end":max_end}



    if min_start and max_end:
        print(f"Language Model: {language_model_id}")
        print(f"Minimum start time: {min_start.isoformat()}")
        print(f"Maximum end time: {max_end.isoformat()}")
        print(f"Total duration: {total_duration}")
    else:
        print("No valid time entries found in the file.")

print("-"*50)
print(json.dumps(token_sums))

  0%|          | 0/8 [00:00<?, ?it/s]

 12%|█▎        | 1/8 [00:37<04:21, 37.34s/it]

Language Model: gpt-4o
Minimum start time: 2024-08-10T11:58:46.061685+00:00
Maximum end time: 2024-08-13T01:11:31.050630+00:00
Total duration: 2 days, 13:12:44.988945


 25%|██▌       | 2/8 [00:55<02:38, 26.35s/it]

Language Model: claude-3-5-sonnet@20240620
Minimum start time: 2024-08-15T15:33:58.174027+00:00
Maximum end time: 2024-08-16T04:11:24.333630+00:00
Total duration: 12:37:26.159603


...

Language Model: claude-3-haiku@20240307
Minimum start time: 2024-08-06T15:06:11.403519+00:00
Maximum end time: 2024-08-07T04:12:42.930302+00:00
Total duration: 13:06:31.526783


 88%|████████▊ | 7/8 [06:23<00:50, 50.03s/it]

Language Model: gemini-1.5-pro-001
Minimum start time: 2024-08-16T18:58:36.547977+00:00
Maximum end time: 2024-08-24T08:12:48.396203+00:00
Total duration: 7 days, 13:14:11.848226


100%|██████████| 8/8 [06:32<00:00, 49.04s/it]

Language Model: meta-llama_Meta-Llama-3.1-70B-Instruct-Turbo
Minimum start time: 2024-08-12T13:43:30.890825+00:00
Maximum end time: 2024-08-12T15:50:00.757549+00:00
Total duration: 2:06:29.866724
--------------------------------------------------
{"gpt-4o": {"input_tokens": 31239943, "output_tokens": 615842}, "claude-3-5-sonnet@20240620": {"input_tokens": 38146081, "output_tokens": 956140}, "open-mistral-nemo": {"input_tokens": 15409072, "output_tokens": 632242}, "gpt-4o-mini": {"input_tokens": 20910559, "output_tokens": 1623186}, "claude-3-haiku@20240307": {"input_tokens": 32364313, "output_tokens": 977883}, "null": {"input_tokens": 5424, "output_tokens": 3}, "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo": {"input_tokens": 10116424, "output_tokens": 109352}}


# Trial Summary for haiku

In [9]:
import numpy as np
import optuna
import os
from tqdm import tqdm
from glob import glob
import pandas as pd
from datetime import datetime, timezone
import ijson
import re
from collections import defaultdict

merged_study_name = "pipeline_claude-3-haiku@20240307_full-supplement"

lm_haiku = "pipeline_claude-3-haiku@20240307"
storage_name = f"sqlite:///{lm_haiku}.db"

def parse_usage(value, model):
    token_pattern = r"Usage\(input_tokens=(\d+), output_tokens=(\d+)\)"
    mistral_pattern = r"UsageInfo\(prompt_tokens=(\d+), total_tokens=(\d+), completion_tokens=(\d+)\)"
    
    if model.startswith('claude'):
        match = re.search(token_pattern, value)
        if match:
            return int(match.group(1)), int(match.group(2))
    elif model.startswith('open-mistral'):
        match = re.search(mistral_pattern, value)
        if match:
            return int(match.group(1)), int(match.group(3))
    elif model.startswith('gpt') or model.startswith('meta_llama'):
        try:
            output_data = json.loads(value)
            usage_data = output_data.get('usage', {})
            return usage_data.get('prompt_tokens', 0), usage_data.get('completion_tokens', 0)
        except json.JSONDecodeError:
            pass
    
    return 0, 0

# Create DataFrame with trial information
trial_data = []
study_summaries = optuna.get_all_study_summaries(storage=storage_name)
for summary in study_summaries:
    if summary.study_name != merged_study_name:
        continue

    study = optuna.load_study(study_name=summary.study_name, storage=storage_name)



    for trial in study.get_trials():

        if trial.datetime_start:
            if not trial.datetime_complete:
                datetime_complete = datetime.now()
            else:
                datetime_complete = trial.datetime_complete
            trial_data.append({
                'study_name': summary.study_name.replace(lm_haiku + "_", ""),
                'trial_number': trial.number,
                'start_time': trial.datetime_start.replace(tzinfo=timezone.utc),
                'end_time': datetime_complete.replace(tzinfo=timezone.utc),
                'duration_hours': (datetime_complete - trial.datetime_start).total_seconds() / 3600,
                'input_tokens': 0,
                'output_tokens': 0,
                'models': set()
            })

trial_df = pd.DataFrame(trial_data)
trial_df = trial_df.sort_values('start_time')

# Create a list of (start_time, end_time, index) tuples for efficient bucketing
trial_intervals = [(row['start_time'], row['end_time'], idx) for idx, row in trial_df.iterrows()]

def bucket_data(item, trial_intervals):
    start_time, end_time, model, input_tokens, output_tokens = item
    buckets = []
    for trial_start, trial_end, trial_idx in trial_intervals:
        if (trial_start <= start_time < trial_end) or (trial_start < end_time <= trial_end) or (start_time <= trial_start < end_time):
            buckets.append(trial_idx)
        # else:
            # print(f"Item {item} does not fit in trial interval {trial_start} - {trial_end}")
    # if len(buckets) > 0:
    #     print(f"Item {item} fits in buckets {buckets}")
    return buckets

def process_json_file(file_path, trial_intervals):
    bucketed_data = defaultdict(list)
    with open(file_path, 'rb') as file:
        parser = ijson.parse(file, multiple_values=True)
        current_start = None
        current_end = None
        current_model = None
        in_model_request = False
        input_tokens = 0
        output_tokens = 0

        for prefix, event, value in parser:
            if prefix.endswith('start_time') and event == 'string':
                current_start = datetime.fromisoformat(value.rstrip('Z')).replace(tzinfo=timezone.utc)
            elif prefix.endswith('end_time') and event == 'string':
                current_end = datetime.fromisoformat(value.rstrip('Z')).replace(tzinfo=timezone.utc)
            
            if prefix.endswith('llm.model_name') and event == 'string':
                current_model = value
                in_model_request = True
            elif prefix.endswith('name') and event == 'string':
                in_model_request = False

            if in_model_request and current_model:
                if prefix.endswith('output.value') and event == 'string':
                    input_tokens, output_tokens = parse_usage(value, current_model)

            if prefix.endswith('attributes.usage') and event == 'string':
                usage_data = json.loads(value)
                input_tokens += usage_data.get('prompt_tokens', 0)
                output_tokens += usage_data.get('completion_tokens', 0)

            # If we have all the necessary information, process and bucket the item
            if current_start and current_end and current_model and (input_tokens > 0 or output_tokens > 0):
                item = (current_start, current_end, current_model, input_tokens, output_tokens)
                for bucket in bucket_data(item, trial_intervals):
                    bucketed_data[bucket].append(item)
                
                # Reset for the next item
                current_start = None
                current_end = None
                current_model = None
                input_tokens = 0
                output_tokens = 0
                in_model_request = False

    return bucketed_data

# Process all JSON files
all_bucketed_data = defaultdict(list)
glob_paths = glob(os.path.join(os.path.abspath(""), "trace_full*.json"))
for file_path in tqdm(glob_paths, desc="Processing and bucketing JSON files"):
    file_data = process_json_file(file_path, trial_intervals)
    for bucket, items in file_data.items():
        all_bucketed_data[bucket].extend(items)

# Function to process bucketed data for a trial
def process_trial_bucket(trial_row, bucket_data):
    trial_start = trial_row['start_time']
    trial_end = trial_row['end_time']
    input_tokens = 0
    output_tokens = 0
    models = set()

    for start, end, model, in_tokens, out_tokens in bucket_data:
        overlap_start = max(trial_start, start)
        overlap_end = min(trial_end, end)
        overlap_ratio = (overlap_end - overlap_start).total_seconds() / (end - start).total_seconds()
        input_tokens += int(in_tokens * overlap_ratio)
        output_tokens += int(out_tokens * overlap_ratio)
        models.add(model)

    return input_tokens, output_tokens, models

# Update trial data
for idx, row in tqdm(trial_df.iterrows(), total=len(trial_df), desc="Processing trials"):
    input_tokens, output_tokens, models = process_trial_bucket(row, all_bucketed_data[idx])
    trial_df.at[idx, 'input_tokens'] = input_tokens
    trial_df.at[idx, 'output_tokens'] = output_tokens
    trial_df.at[idx, 'models'] = models

# Convert set of models to string
trial_df['models'] = trial_df['models'].apply(lambda x: ', '.join(sorted(x)))

Processing trials: 100%|██████████| 10/10 [00:00<00:00, 154.17it/s]


In [10]:
import pandas as pd

merged_study_name = "pipeline_claude-3-haiku@20240307_full-supplement"

merged_study = optuna.load_study(study_name=merged_study_name, storage=storage_name)
# param_importances = optuna.importance.get_param_importances(merged_study, evaluator=optuna.importance.PedAnovaImportanceEvaluator())
param_importances = optuna.importance.get_param_importances(merged_study)


# Create DataFrame for parameter importances
df_importances = pd.DataFrame(list(param_importances.items()), columns=['Parameter', 'Importance'])

# Extract trial data for each parameter
trial_data = {f"Trial {i}" if i > 0 else "Baseline": trial.params for i, trial in enumerate(merged_study.trials)}

# Create a DataFrame from the trial data
df_trials = pd.DataFrame(trial_data)

# Now, transpose the trial data DataFrame so that parameters are the rows and trials are columns
df_trials_transposed = df_trials.T

# Merge the importance DataFrame with the transposed trial data
df_merged = df_importances.set_index('Parameter').join(df_trials_transposed.T)

# Display the merged DataFrame
# print(df_merged)


latex_table_with_index = df_merged.reset_index().to_latex(index=False, float_format="%.6f", column_format="llcccc", longtable=False, caption="Parameter importances and trial values", label="tab:param_importances_and_trials", position="H")

latex_table_with_index = latex_table_with_index.replace("_", "\\_")


latex_table_with_index = latex_table_with_index.replace("lspCheck", "LSP Check")
latex_table_with_index = latex_table_with_index.replace("codeType", "Code")
latex_table_with_index = latex_table_with_index.replace("errorType", "Mvn Error")
latex_table_with_index = latex_table_with_index.replace("apiChangeType","API Change")
latex_table_with_index = latex_table_with_index.replace("dependencyChangeType", "Dep. Change")
latex_table_with_index = latex_table_with_index.replace("max\\_hops", "Max. Hops")

latex_table_with_index = latex_table_with_index.replace("MINIFIED\\_PARSED & MINIFIED\\_PARSED & MINIFIED\\_PARSED & MINIFIED\\_PARSED", "\\multicolumn{4}{c}{MINIFIED\\_PARSED}")
latex_table_with_index = latex_table_with_index.replace("REVAPI & REVAPI & REVAPI & REVAPI", "\\multicolumn{4}{c}{REVAPI}")
latex_table_with_index = latex_table_with_index.replace("0.000000", "0")


# Display the LaTeX table
print(latex_table_with_index)

\begin{table}[H]
\caption{Parameter importances and trial values}
\label{tab:param\_importances\_and\_trials}
\begin{tabular}{llcccc}
\toprule
Parameter & Importance & Baseline & Trial 1 & Trial 2 & Trial 3 & Trial 4 & Trial 5 & Trial 6 & Trial 7 & Trial 8 & Trial 9 \\
\midrule
API Change & 0.610320 & \multicolumn{4}{c}{REVAPI} & REVAPI & OMIT & \multicolumn{4}{c}{REVAPI} \\
Mvn Error & 0.209128 & MINIFIED & MINIFIED & SUPER\_MINIFIED & MINIFIED & MINIFIED & MINIFIED & MINIFIED & MINIFIED & OMIT & MINIFIED \\
Code & 0.064068 & ALL & ALL & ALL & MINIFIED & ALL & ALL & ALL & MINIFIED & ALL & ALL \\
Max. Hops & 0.062674 & 30 & 30 & 30 & 30 & 40 & 30 & 30 & 30 & 10 & 30 \\
Dep. Change & 0.034661 & \multicolumn{4}{c}{MINIFIED\_PARSED} & MINIFIED\_PARSED & MINIFIED\_PARSED & DIFF & OMIT & DIFF & MINIFIED\_PARSED \\
LSP Check & 0.019149 & False & True & False & False & False & False & False & True & False & True \\
\bottomrule
\end{tabular}
\end{table}



In [11]:
def format_number(x: str, round_to_millions=False):
    if pd.isna(x) or x == 'nan' or x == '-' or x == '' or x is None:
        return '-'
    
    try:
        # Remove commas and convert to float first
        num = float(str(x).replace(',', ''))
        
        if round_to_millions:
                
            # Round to nearest million and format
            millions = round(num / 1_000_000, 3)


            if num < 1_000_000:
                # Round to nearest thousand and format
                thousands = round(num / 1_000, 1)
                if thousands == 0:
                    return '-'
                if thousands.is_integer():
                    return f'{int(thousands)}K'
                return f'{thousands:.1f}K'

            if millions == 0:
                return '-'
            if millions.is_integer():
                return f'{int(millions)}M'
            return f'{millions:.3f}M'
        
        
        
    except ValueError:
        # If conversion fails, return '-'
        return '-'

trial_df_formatted = trial_df.copy()

trial_df_formatted.loc[0,'input_tokens'] = token_sums.get('claude-3-haiku@20240307', {"input_tokens": 0})['input_tokens']
trial_df_formatted.loc[0,'output_tokens'] = token_sums.get('claude-3-haiku@20240307', {"output_tokens": 0})['output_tokens']
    
trial_df_formatted['input_tokens'] = trial_df_formatted['input_tokens'].apply(lambda x: format_number(x, round_to_millions=True))
trial_df_formatted['output_tokens'] = trial_df_formatted['output_tokens'].apply(lambda x: format_number(x, round_to_millions=True)) 


trial_df_formatted = trial_df_formatted.drop('models', axis=1)



trial_df_formatted

,study_name,trial_number,start_time,end_time,duration_hours,input_tokens,output_tokens
0,full-supplement,0,2024-08-06 15:06:11.324844+00:00,2024-08-07 04:12:42.947245+00:00,13.108784,32.364M,977.9K
1,full-supplement,1,2024-08-13 11:39:42.897107+00:00,2024-08-14 19:54:09.881340+00:00,32.240829,30.355M,953.4K
2,full-supplement,2,2024-08-14 19:54:09.968593+00:00,2024-08-15 04:28:21.591849+00:00,8.569895,32.184M,1.149M
3,full-supplement,3,2024-08-15 04:28:21.655739+00:00,2024-08-15 11:16:08.628274+00:00,6.796381,25.808M,1.153M
4,full-supplement,4,2024-08-19 12:13:05.196101+00:00,2024-08-20 01:10:28.716932+00:00,12.956534,43.068M,1.277M
5,full-supplement,5,2024-08-20 01:10:28.797313+00:00,2024-08-20 09:16:17.257809+00:00,8.096795,27.891M,1.485M
6,full-supplement,6,2024-08-20 09:16:17.308355+00:00,2024-08-20 19:24:21.644748+00:00,10.134538,41.004M,1.432M
7,full-supplement,7,2024-08-20 19:24:21.719026+00:00,2024-08-22 04:26:44.347626+00:00,33.039619,23.255M,1.086M
8,full-supplement,8,2024-08-22 04:26:44.405607+00:00,2024-08-22 09:47:19.490838+00:00,5.343079,10.179M,434.1K
9,full-supplement,9,2024-08-22 09:47:19.545838+00:00,2024-08-22 12:11:14.601729+00:00,2.398627,3.789M,53.6K


In [36]:

import pandas as pd
import matplotlib.pyplot as plt




# Transpose and create DataFrame
df = pd.DataFrame(duration_by_language_model).T

# Convert columns to datetime
df['min_start'] = pd.to_datetime(df['min_start'])
df['max_end'] = pd.to_datetime(df['max_end'])


df.loc['gemini-1.5-pro-001', 'min_start'] = pd.to_datetime("2024-08-23 15:24:04.674398+00:00")
df.loc['gemini-1.5-pro-001', 'max_end'] = pd.to_datetime("2024-08-24 08:12:48.414666+00:00")



# Calculate duration and add new column
df['duration'] = df['max_end'] - df['min_start']

import optuna
storage_name = "sqlite:///{}.db".format(f"pipeline_gpt-4o")
study_summaries = optuna.get_all_study_summaries(storage=storage_name)
start_times = []
end_times = []

total_duration = pd.Timedelta(0)

for summary in study_summaries:
    study = optuna.load_study(study_name=summary.study_name, storage=storage_name)
    trials = study.get_trials(deepcopy=False)
    for trial in trials:
        if trial.datetime_start and trial.datetime_complete:
            total_duration += trial.datetime_complete - trial.datetime_start

# Due to budget overruns, we split the trials in 2
df.loc['gpt-4o', 'duration'] = total_duration



df['duration_secs'] = df['duration'].dt.total_seconds()
df['duration_hours'] = df['duration'].dt.total_seconds() / 3600

# replace index by get_language_model
df.index = df.index.map(raw_model_mapper)


# Give Index the name "LM"
df.index.name = "LM"

gemini_i_short = 68_374_823
gemini_i_long = 51_114_988

gemini_o_short = 1_361_327
gemini_o_long = 84_286

metadata = [
    {
        "LM":"mistral-nemo",
        "total_input_tokens": 991_625+25_510_617,
        "total_output_tokens": 8_496+896_669,
    },
    {
        "LM":"gpt-4o-mini",
        "total_input_tokens": token_sums["gpt-4o-mini"]['input_tokens'],
        "total_output_tokens": token_sums["gpt-4o-mini"]['output_tokens'],
    },
    {
        "LM":"gemini-1.5-pro",
        "total_input_tokens_short": gemini_i_short,
        "total_output_tokens_short": gemini_o_short,
        "total_input_tokens_long": gemini_i_long,
        "total_output_tokens_long": gemini_o_long,
        "total_input_tokens": gemini_i_short + gemini_i_long,
        "total_output_tokens": gemini_o_short + gemini_o_long,
    },
    {
        "LM":"gpt-4o",
        "total_input_tokens": token_sums["gpt-4o"]['input_tokens'],
        "total_output_tokens": token_sums["gpt-4o"]['output_tokens'],
    },
    {
        "LM":"text-bison-002",
        "total_input_tokens": 20_501_428,
        "total_output_tokens": 2_365_535,
    },
    {
        "LM":"claude-3-haiku",
        "total_input_tokens": token_sums["claude-3-haiku@20240307"]['input_tokens'],
        "total_output_tokens": token_sums["claude-3-haiku@20240307"]['output_tokens'],
    },
    {
        "LM":"claude-3.5-sonnet",
        "total_input_tokens": token_sums["claude-3-5-sonnet@20240620"]['input_tokens'],
        "total_output_tokens": token_sums["claude-3-5-sonnet@20240620"]['output_tokens'],
    },
    {
        "LM":"Llama-3.1-70B",
        "total_input_tokens": token_sums["meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"]['input_tokens'],
        "total_output_tokens": token_sums["meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"]['output_tokens'],
    }
]

# merge in 
df = df.merge(pd.DataFrame(metadata), on="LM")
# print(df)

# # Data sourced from billing platforms
# df.loc['mistral-nemo', 'total_input_tokens'] = 25_510_617
# df.loc['mistral-nemo', 'total_output_tokens'] = 896_669

# df.loc['gpt-4o-mini', 'total_input_tokens'] = 36_627_622
# df.loc['gpt-4o-mini', 'total_output_tokens'] = 1_240_294

# df.loc['text-bison-002', 'total_input_tokens'] = 20_501_428
# df.loc['text-bison-002', 'total_output_tokens'] = 2_365_535


# df.loc['claude-3-haiku', 'total_input_tokens'] = -1
# df.loc['claude-3-haiku', 'total_output_tokens'] = -1

# df.loc['Llama-3.1-70B', 'total_input_tokens'] = -1
# df.loc['Llama-3.1-70B', 'total_output_tokens'] = -1

df


,LM,min_start,max_end,duration,duration_secs,duration_hours,total_input_tokens,total_output_tokens,total_input_tokens_short,total_output_tokens_short,total_input_tokens_long,total_output_tokens_long
0,gpt-4o,2024-08-10 11:58:46.061685+00:00,2024-08-13 01:11:31.050630+00:00,0 days 17:26:40.373620,62800.373620,17.444548,31239943,615842,NaN,NaN,NaN,NaN
1,claude-3.5-sonnet,2024-08-15 15:33:58.174027+00:00,2024-08-16 04:11:24.333630+00:00,0 days 12:37:26.159603,45446.159603,12.623933,38146081,956140,NaN,NaN,NaN,NaN
2,mistral-nemo,2024-08-02 09:56:20.759045+00:00,2024-08-02 14:57:02.718129+00:00,0 days 05:00:41.959084,18041.959084,5.011655,26502242,905165,NaN,NaN,NaN,NaN
3,gpt-4o-mini,2024-08-01 20:49:30.989272+00:00,2024-08-02 09:28:41.812181+00:00,0 days 12:39:10.822909,45550.822909,12.653006,20910559,1623186,NaN,NaN,NaN,NaN
4,text-bison-002,2024-08-05 11:57:47.277504+00:00,2024-08-05 14:15:10.512269+00:00,0 days 02:17:23.234765,8243.234765,2.289787,20501428,2365535,NaN,NaN,NaN,NaN
5,claude-3-haiku,2024-08-06 15:06:11.403519+00:00,2024-08-07 04:12:42.930302+00:00,0 days 13:06:31.526783,47191.526783,13.108757,32364313,977883,NaN,NaN,NaN,NaN
6,gemini-1.5-pro,2024-08-23 15:24:04.674398+00:00,2024-08-24 08:12:48.414666+00:00,0 days 16:48:43.740268,60523.740268,16.812150,119489811,1445613,68374823.0,1361327.0,51114988.0,84286.0
7,Llama-3.1-70B,2024-08-12 13:43:30.890825+00:00,2024-08-12 15:50:00.757549+00:00,0 days 02:06:29.866724,7589.866724,2.108296,10116424,109352,NaN,NaN,NaN,NaN


In [37]:
new_df = df.copy()

# Move index into a column "LM"
new_df.reset_index(inplace=True)

print(new_df.columns)

new_df['total_tokens'] = new_df['total_input_tokens'] + new_df['total_output_tokens']
# df['total_tokens'] = df['total_tokens'].apply(lambda x: f"{x:,}")

# new_df["DSPY Compile Success"] = 0
# new_df["DSPY Test Success"] = 0



# merged_df = new_df.merge(success_df, left_on='LM', right_on='LM', how='left')
merged_df = new_df.merge(success_df, how="right", on="LM")
# print(merged_df)
# merged_df['DSPY Compile Success'] = (merged_df['Compile Success'] / merged_df['Attempts']).apply(lambda x: f"{x:.2%}" if not pd.isna(x) else '-')
# merged_df['DSPY Test Success'] = (merged_df['Test Success'] / merged_df['Attempts']).apply(lambda x: f"{x:.2%}" if not pd.isna(x) else '-')

print(merged_df.columns)

merged_df.rename(columns={'Test Success': 'DSPY Test Success', 'Compile Success': 'DSPY Compile Success'}, inplace=True)

merged_df['DSPY Errors'] = merged_df['Errors']
merged_df['DSPY Attempts'] = merged_df['Attempts']



# LaTex Table with LM and duration_hours
# final_df = merged_df[['LM', 'duration_hours', 'total_tokens', 'DSPY Compile Success', 'DSPY Test Success']]
final_df = merged_df.rename(columns={'duration_hours': 'DSPY Duration (hours)', 'total_tokens': 'DSPY Total Tokens', 
                                     'total_input_tokens': 'Input Tokens DSPY', 'total_output_tokens': 'Output Tokens DSPY','total_input_tokens_short': 'Input Tokens Short DSPY',
'total_output_tokens_short': 'Output Tokens Short DSPY',
'total_input_tokens_long': 'Input Tokens Long DSPY',
'total_output_tokens_long': 'Output Tokens Long DSPY'})


final_df = final_df[["LM",
                     "Attempts",
"DSPY Duration (hours)",
"DSPY Total Tokens",
"DSPY Compile Success",
"DSPY Test Success",
"DSPY Errors",
'Input Tokens DSPY',
'Output Tokens DSPY',
'Input Tokens Short DSPY',
'Output Tokens Short DSPY',
'Input Tokens Long DSPY',
'Output Tokens Long DSPY',
                     ]]

final_df = final_df.fillna(0)
print(final_df.to_json(orient='records', indent=4))

Index(['index', 'LM', 'min_start', 'max_end', 'duration', 'duration_secs',
       'duration_hours', 'total_input_tokens', 'total_output_tokens',
       'total_input_tokens_short', 'total_output_tokens_short',
       'total_input_tokens_long', 'total_output_tokens_long'],
      dtype='object')
Index(['index', 'LM', 'min_start', 'max_end', 'duration', 'duration_secs',
       'duration_hours', 'total_input_tokens', 'total_output_tokens',
       'total_input_tokens_short', 'total_output_tokens_short',
       'total_input_tokens_long', 'total_output_tokens_long', 'total_tokens',
       'Attempts', 'Errors', 'Compile Success', 'Test Success'],
      dtype='object')
[
    {
        "LM":"claude-3.5-sonnet",
        "Attempts":65,
        "DSPY Duration (hours)":12.6239332231,
        "DSPY Total Tokens":39102221,
        "DSPY Compile Success":28,
        "DSPY Test Success":12,
        "DSPY Errors":25,
        "Input Tokens DSPY":38146081,
        "Output Tokens DSPY":956140,
        "Input

In [ ]:
latex_table = final_df.to_latex(
    index=False,
    float_format="%.2f", 
    caption="Duration of Augmented Prompt Execution by Language Model", 
    label="tab:augmented-prompt-duration", 
    position="H"
)

print(latex_table)

In [ ]:
# find the hop value via the hops dict for the commit hashes in compiled-hash and full_successes-hash in trial_data for each trial
# "hops": {
#             "6c53cd904bd66fc79af8687571e607c259226b81": 10,
#             "8502e85f9ee2ff90ce96b47b5904f011e81e8bb8": 10,
#             "489aad6060454d0b7b34a144e0b345c5a3a199f5": 10,
#             "165381d26b2c3d2278fde88c16f95807506451fe": 10,
#             "8fbb6deb112102ef7507a8e68c5215e5f481d03b": 10,
#             "250cafc7d6ae47d5d4803b5a5e58186eb81fa3b5": 10,
#             "c32185c43be158d32c7d13c5b816991954eb45fa": 10,
#             "2d733a58045b4bf3669aa00d875e77f9db48c29b": 10,
#             "dc9f7910968cd0aa2090e390045ae053693e839a": 10,
#             "a2b0fc53611f8705640773f18c8dd6a47eed3b7f": 10,
#             "a4c360001134c2e3a9f7fbde88a07a9fd767e78e": 10,
#             "ad80bdff62b1b0520d3fb9e8d627532a38a7c60c": 10,
#             "ea33b5101edffc0242967cbf21c1016378b18483": 10,
#             "1d43bce1de6a81ac017c233d72f348d3c850299e": 10,
#             "979d6237a50840cd925cc1a33c415ffbbbc42846": 10,
#             "af6e5d1cc94f031f29b4838e7a8b56704c8c5de4": 10,
#             "d3af06df4613be146bb9f8034e1a8a3098050c82": 10,
#             "c7c9590a206d4fb77dd05b9df391d888e6181667": 10,
#             "14fc5fa696f499cac48401b3a86882b3bf7d9b82": 10,
#             "b6a48a6e557fad1ceda680618e0a34c7b8c5c087": 10,
#             "5fcd0c3ad7727850c47602b17530dc355e5bd097": 10,
#             "3ff575ae202cdf76ddfa8a4228a1711a6fa1e921": 10,
#             "4b4c08d502d98d240855013ab76008f5e0243435": 10,
#             "3572a1ecc0154c61e05505aed56055b9c5e539a6": 10,
#             "acc50dabec6796c091b84c1ada2ae4cbcab8b562": 10
#         },

hop_values = []
for index, row in trial_data.items():
  print(row)
  compiled_hashes = row.get('compiled-hash', None)
  full_successes_hashes = row.get('full_successes-hash', None)
  hops = row['hops']

  # print(compiled_hashes, full_successes_hashes)

  # for hash in compiled_hashes:
  #   print(hops.get(hash))
  
  compiled_hop = hops.get(compiled_hashes[0]) if compiled_hashes else None
  full_successes_hop = hops.get(full_successes_hashes[0]) if full_successes_hashes else None
  
  hop_values.append((compiled_hop, full_successes_hop))
  
hop_values

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


df = pd.DataFrame.from_dict(trial_data, orient='index')

# Sorting by trial number
df_sorted = df.sort_index()
df_sorted.index = df_sorted.index.astype(int)
df_sorted = df_sorted.sort_index()

# Plotting the data as a bar chart
fig, ax = plt.subplots()

# Plotting 'compiled' and 'full_successes' as bar charts
df_sorted[['compiled', 'full_successes', 'tests_broken_in_environment', 'no_repro']].plot(kind='bar', ax=ax)

# Setting labels and title
ax.set_xlabel('Trial Number')
ax.set_ylabel('Count')
ax.set_title(f'Compiled and Full Successes by Trial Number {language_model_id}')

ax.yaxis.get_major_locator().set_params(integer=True)

plt.xticks(rotation=0)

# Adding legend
ax.legend(['Compiled (total)', 'Test Suite completed', 'Test Suite broken in environment', 'no_repro'])

plt.show()

In [ ]:
# Plotting the data with 'attempts_not_compiled' always on top as a bar chart
fig, ax = plt.subplots(figsize=(12, 8))

# Reordering the columns to have 'attempts_not_compiled' on top
df_sorted_reordered = df_sorted[['full_successes', 'tests_broken_in_environment', 'no_repro', 'attempts_not_compiled']]

# Plotting the data
df_sorted_reordered.plot(kind='bar', stacked=True, ax=ax)

# Setting labels and title
ax.set_xlabel('Trial Number')
ax.set_ylabel('Count')
ax.set_title(f'Compilation Attempts and Results (for valid diff attempts by the LLM) by Trial Number for {language_model_id}')

plt.xticks(rotation=0)

# Adding legend
ax.legend(['Full Successes', 'Broken Tests', 'No Repro', 'Failed Compilation Attempts'])

# Ensure the y-axis only shows full numbers
ax.yaxis.get_major_locator().set_params(integer=True)

# Adding data labels, only showing numbers if not 0
for container in ax.containers:
    labels = [f'{v:.0f}' if v != 0 else '' for v in container.datavalues]
    ax.bar_label(container, labels=labels, label_type='center')

plt.tight_layout()
plt.show()

In [ ]:
# df_sorted = pd.DataFrame.from_dict(trial_data, orient='index')
error_details_columns = [col for col in df_sorted.columns if col.startswith('error_details_')]

# Plotting the data with error details as a stacked bar chart
fig, ax = plt.subplots(figsize=(12, 8))

# Plotting error details as stacked bar charts
df_sorted[error_details_columns].plot(kind='bar', stacked=True, ax=ax)

# Setting labels and title
ax.set_xlabel('Trial Number')
ax.set_ylabel('Count')
ax.set_title(f'Pre-Compilation Error Details by Trial Number for {language_model_id}')

plt.xticks(rotation=0)

# Adding legend
ax.legend([col.replace('error_details_', '').replace('_', ' ').title() for col in error_details_columns])

# Ensure the y-axis only shows full numbers
ax.yaxis.get_major_locator().set_params(integer=True)

# Adding data labels, only showing numbers if not 0
for container in ax.containers:
    labels = [f'{v:.0f}' if v != 0 else '' for v in container.datavalues]
    ax.bar_label(container, labels=labels, label_type='center')

plt.tight_layout()
plt.show()

In [ ]:
# test for error prompt tokens

# Research Question: Are errors persistent between the executions?

In [ ]:
import optuna


language_model_id = "claude-3-haiku@20240307"
trial = "full_supplement"
study_name = "pipeline_" + f"{language_model_id}_"+trial
print("Loading study", study_name)
storage_name = "sqlite:///{}.db".format(f"pipeline_{language_model_id}")
study = optuna.load_study(storage=storage_name, study_name=study_name)

In [ ]:
import optuna

# Load the two studies
study1 = optuna.load_study(study_name="pipeline_claude-3-haiku@20240307_dspy-baseline", storage=storage_name)
study2 = optuna.load_study(study_name="pipeline_claude-3-haiku@20240307_full_supplement", storage=storage_name)


merged_study_name = "pipeline_claude-3-haiku@20240307_merged_study"

if merged_study_name not in optuna.get_all_study_names(storage_name):

    # Create a new study to merge them
    merged_study = optuna.create_study(study_name=merged_study_name, direction="maximize", storage=storage_name)

    # Function to add trials from one study to another
    def add_trials_to_study(source_study, target_study):
        for trial in source_study.trials:
            target_study.add_trial(trial)

    # Merge the trials from both studies into the new one
    add_trials_to_study(study1, merged_study)
    add_trials_to_study(study2, merged_study)

else:
    # Load the merged study
    merged_study = optuna.load_study(study_name=merged_study_name, storage=storage_name)

# optuna.study.save_study_to_storage(merged_study)

# Get the new parameter importances for the merged study

# Display the parameter importances
print(param_importances)


In [ ]:
optuna.visualization.plot_intermediate_values(study)

# Diff Analysis

In [11]:
from collections import defaultdict
from pathlib import Path
import os
import json
import pandas as pd

diffs = defaultdict(dict) 
data_path = Path(os.path.abspath(""))/"dataset"
import tempfile
from unidiff import PatchSet

from masterthesis.agent.GitAgent import GitAgent

git_slugs = defaultdict(str)

for file in data_path.glob(f"*/out/dspy-baseline-*-execution-errors.json"):
  lm = file.parts[-1].replace("-execution-errors.json", "").replace("dspy-baseline-", "")
  commit = file.parts[-3]
  with open(file, "r") as f:
    execution_errors = json.load(f)
    if "output" in execution_errors:
      if len(execution_errors["output"]) == 1:
        diff = execution_errors["output"][0]
        # diff = diff.replace('```diff', '').replace('```', '')
        if commit not in diffs[lm]:
              diffs[lm][commit] = set()
        diffs[lm][commit].add(diff)
        git_slugs.update({commit: execution_errors["input"]["repo_slug"]})




from unidiff import PatchSet

from masterthesis.agent.aider.AdvancedDiffAgent import UnifiedDiffCoder

# Function to calculate diff statistics
def calculate_diff_stats(diff, commit, language_model):

    repo_path = data_path / f"{commit}/repo"
    git_agent = GitAgent(Path(repo_path), commit_hash=commit, github_slug=git_slugs[commit])
    git_agent.discard_changes()
    coder = UnifiedDiffCoder(repo_dir=repo_path)
    for diff_text in [diff]:
        paths = coder.get_paths(diff_text)
        for path in paths:

            output_location = Path(repo_path) / path
            is_valid_diff, patched_file = coder.apply_edits(diff_text)
            # assert is_valid_diff, f"Diff for {path} is not valid"

            if is_valid_diff:

                with open(output_location, "w", encoding="utf-8") as out_file_wrapper:
                    out_file_wrapper.write(patched_file)
    full_diff = git_agent.get_full_diff()
    with open(data_path / commit / f"{language_model}.diff", "w") as f:
        f.write(full_diff)
    patch = PatchSet(full_diff)
    git_agent.discard_changes()
        

    
    operation_count = sum([file_patch.added+file_patch.removed for file_patch in patch])
    hunk_count = sum([len(file_patch) for file_patch in patch])

    return {
        'files_modified': len(patch),  
        'operation_count': operation_count,
        'hunk_count': hunk_count
    }

# Transform diffs and add statistics
transformed_diffs = defaultdict(lambda: defaultdict(list))
for lm, commits in diffs.items():
    for commit, diff_set in commits.items():
        for diff in diff_set:
            stats = calculate_diff_stats(diff, commit, lm)
            transformed_diffs[lm][commit].append({
                'diff': diff,
                'files_modified': stats['files_modified'],
                'operation_count': stats['operation_count'],
                'hunk_count': stats['hunk_count']
            })

# Create a DataFrame
rows = []
for lm, commits in transformed_diffs.items():
    for commit, diffs in commits.items():
        for diff_data in diffs:
            rows.append({
                'language_model': lm,
                'commit': commit,
                'files_modified': diff_data['files_modified'],
                'operation_count': diff_data['operation_count'],
                'hunk_count': diff_data['hunk_count']
            })

diff_df = pd.DataFrame(rows)

# Commit-level statistics
commit_stats = diff_df.groupby(['language_model', 'commit']).agg({
    'files_modified': ['mean', 'median', 'std'],
    'operation_count': ['mean', 'median', 'std'],
    'hunk_count': ['mean', 'median', 'std']
}).reset_index()

# Language model comparison
lm_comparison = diff_df.groupby('language_model').agg({
    'files_modified': ['mean', 'median', 'std'],
    'operation_count': ['mean', 'median', 'std'],
    'hunk_count': ['mean', 'median', 'std']
}).reset_index()

# diff_df.to_pickle("diff_stats.pkl")

lm_comparison

Repository is dirty. Discarding changes...
All changes have been discarded.
Repository is dirty. Discarding changes...
All changes have been discarded.
Repository is dirty. Discarding changes...
All changes have been discarded.
Repository is dirty. Discarding changes...
All changes have been discarded.
Repository is dirty. Discarding changes...
All changes have been discarded.
Repository is dirty. Discarding changes...
All changes have been discarded.
Repository is dirty. Discarding changes...
All changes have been discarded.
Repository is dirty. Discarding changes...
All changes have been discarded.
Repository is dirty. Discarding changes...
All changes have been discarded.
Repository is dirty. Discarding changes...
All changes have been discarded.
Repository is dirty. Discarding changes...
All changes have been discarded.
Repository is dirty. Discarding changes...
All changes have been discarded.
Repository is dirty. Discarding changes...
All changes have been discarded.
Repository i

language_model files_modified              \
                                                          mean median  std   
0                    claude-3-5-sonnet@20240620            1.0    1.0  0.0   
1                       claude-3-haiku@20240307            1.0    1.0  0.0   
2                            gemini-1.5-pro-001            1.0    1.0  0.0   
3                                        gpt-4o            1.0    1.0  0.0   
4                                   gpt-4o-mini            1.0    1.0  0.0   
5  meta-llama_Meta-Llama-3.1-70B-Instruct-Turbo            1.0    1.0  0.0   
6                             open-mistral-nemo            1.0    1.0  0.0   
7                                text-bison@002            1.0    1.0  0.0   

  operation_count                  hunk_count                   
             mean median       std       mean median       std  
0        3.833333    4.0  1.585923   1.750000    2.0  0.621582  
1        3.142857    3.0  0.899735   1.857143    2.0  0.690066  
2        2.800000    2.0  1.475730   1.100000    1.0  0.316228  
3        4.125000    3.5  2.695896   1.625000    2.0  0.517549  
4        3.750000    4.0  1.258306   2.000000    2.0  0.816497  
5        2.250000    2.0  0.500000   1.000000    1.0  0.000000  
6        3.333333    3.0  0.577350   1.333333    1.0  0.577350  
7        6.000000    6.0  2.828427   1.500000    1.5  0.707107

In [14]:
import numpy as np

lm_comparison_human = lm_comparison.rename(columns={'files_modified': 'Files Modified', 'operation_count': 'Operation Count', 'language_model': "Language Model"})

# print(lm_comparison_human)

def convert_float_to_int(value):
    # if isinstance(value, float):
    #     print(value, value.is_integer())
    if isinstance(value, float) and value.is_integer():
        return str(int(value))
    return value
# lm_comparison_human = lm_comparison_human.map(convert_float_to_int)



def format_mean_std_median(mean, std, median):
    if np.isnan(mean) or np.isnan(std) or np.isnan(median):
        return "N/A"
    
    if mean.is_integer():
        mean_str = str(int(mean))
    else:
        mean_str = f"{mean:.2f}"
    mean_std = f"{mean_str} ± {std:.2f}"
    if mean.is_integer() and std == 0:
        mean_std = f"{int(mean)}"
    if median.is_integer():
        median_str = str(int(median))
    else:
        median_str = f"{median:.2f}"
    return f"{mean_std} ({median_str})"

# Create a new DataFrame for the formatted data
formatted_data = []

for lm, row in lm_comparison.iterrows():
    formatted_row = {
        'Language Model': raw_model_mapper(row["language_model"].to_string().strip()),
        'Files Modified': format_mean_std_median(row['files_modified']['mean'], row['files_modified']['std'], row['files_modified']['median']),
        'Operation Count': format_mean_std_median(row['operation_count']['mean'], row['operation_count']['std'], row['operation_count']['median']),
        'Hunk Count': format_mean_std_median(row['hunk_count']['mean'], row['hunk_count']['std'], row['hunk_count']['median'])
    }
    formatted_data.append(formatted_row)

formatted_df = pd.DataFrame(formatted_data)

lm_order = success_df['LM'].tolist()
formatted_df = formatted_df[formatted_df['Language Model'].isin(lm_order)]
formatted_df['LM'] = pd.Categorical(formatted_df['Language Model'], categories=lm_order, ordered=True)
formatted_df = formatted_df.sort_values('LM').drop(columns=['LM'])



latex_text = formatted_df.to_latex(index=False, escape=False, float_format="%.2f", position="H", caption="Diff Statistics by Language Model Zero Shot", label="tab:diff-stats-zero-shot")
                                          # , column_format="lrrr|rrr")

latex_text = latex_text.replace("±", "$\pm$")
latex_text = latex_text.replace(".50", ".5")



print(latex_text)

\begin{table}[H]
\caption{Diff Statistics by Language Model Zero Shot}
\label{tab:diff-stats-zero-shot}
\begin{tabular}{llll}
\toprule
Language Model & Files Modified & Operation Count & Hunk Count \\
\midrule
claude-3.5-sonnet & 1 (1) & 3.83 $\pm$ 1.59 (4) & 1.75 $\pm$ 0.62 (2) \\
gemini-1.5-pro & 1 (1) & 2.80 $\pm$ 1.48 (2) & 1.10 $\pm$ 0.32 (1) \\
gpt-4o & 1 (1) & 4.12 $\pm$ 2.70 (3.5) & 1.62 $\pm$ 0.52 (2) \\
claude-3-haiku & 1 (1) & 3.14 $\pm$ 0.90 (3) & 1.86 $\pm$ 0.69 (2) \\
Llama-3.1-70B & 1 (1) & 2.25 $\pm$ 0.5 (2) & 1 (1) \\
gpt-4o-mini & 1 (1) & 3.75 $\pm$ 1.26 (4) & 2 $\pm$ 0.82 (2) \\
mistral-nemo & 1 (1) & 3.33 $\pm$ 0.58 (3) & 1.33 $\pm$ 0.58 (1) \\
\bottomrule
\end{tabular}
\end{table}



<>:58: SyntaxWarning: invalid escape sequence '\p'
<>:58: SyntaxWarning: invalid escape sequence '\p'
/tmp/ipykernel_632508/1536726878.py:58: SyntaxWarning: invalid escape sequence '\p'
  latex_text = latex_text.replace("±", "$\pm$")


In [ ]:
dspy_hashes = []
data_path = Path(os.path.abspath("")) / "dataset"
for file in data_path.glob("*"):
    if file.is_dir():
        dspy_hashes.append(file.parts[-1])
dspy_hashes

['489aad6060454d0b7b34a144e0b345c5a3a199f5',
 'c7c9590a206d4fb77dd05b9df391d888e6181667',
 '8502e85f9ee2ff90ce96b47b5904f011e81e8bb8',
 '90ffd2cd31edecf778d14d0015da9ceab7e53081',
 '3ff575ae202cdf76ddfa8a4228a1711a6fa1e921',
 'f26cd85b97b24c07a2e446f43ac8793619fa0724',
 '5fcd0c3ad7727850c47602b17530dc355e5bd097',
 'ea33b5101edffc0242967cbf21c1016378b18483',
 'a80dac86d1caa3958c45c036d93a7d9231d88fbf',
 '1d43bce1de6a81ac017c233d72f348d3c850299e',
 '0a11c04038eae517540051dbf51f7f26b7221f20',
 'a2b0fc53611f8705640773f18c8dd6a47eed3b7f',
 '1ef97ea6c5b6e34151fe6167001b69e003449f95',
 'd3af06df4613be146bb9f8034e1a8a3098050c82',
 'c0f6ab75784dbc13ae8ff47298704c0756cf3a2c',
 'dcc95f410847ab308db2f2a31ab13e32dc65c670',
 'ae16b526695fe275ab5e6a1992916875d26da860',
 '65200df71d5f6ab1c5502f74a5dc7bcbda459563',
 'cbcafe129e143ef09401470e9d11de9758f298d0',
 '36859167815292f279e570d39dd2ddbcf1622dc6',
 '0abf7148300f40a1da0538ab060552bca4a2f1d8',
 '874ed893a4e46ea5182be2be054715967e58f08f',
 'b6a48a6e

In [ ]:
import time
import os
import shutil
import git
import tempfile
import subprocess
import platform
import filecmp
import stat

def clone_repo(github_slug, commit_hash):
    with tempfile.TemporaryDirectory(delete=False) as repo_path:
        start_time = time.time()
        repo = git.Repo.clone_from(f"https://github.com/{github_slug}.git", repo_path)
        repo.git.fetch("origin", commit_hash, depth=2)
        repo.git.checkout(commit_hash)
        end_time = time.time()
        return end_time - start_time, repo_path

def copy_folder(source_path, commit_hash):
    with tempfile.TemporaryDirectory(delete=False) as dest_path:
        start_time = time.time()
        shutil.copytree(source_path, dest_path, symlinks=True, ignore=None, dirs_exist_ok=True)
        end_time = time.time()
        return end_time - start_time, dest_path

def get_directory_hash(directory):
    if platform.system() == "Darwin":  # macOS
        hash_command = ["find", directory, "-type", "f", "-not", "-path", "*/\.*", "-print0", "|", "xargs", "-0", "shasum", "-a", "256", "|", "LC_ALL=C", "sort", "-k", "2", "|", "shasum", "-a", "256"]
    else:  # Linux and other Unix-like systems
        hash_command = ["find", directory, "-type", "f", "-not", "-path", "*/\.*", "-print0", "|", "LC_ALL=C", "sort", "-z", "|", "xargs", "-0", "sha256sum", "|", "sha256sum"]
    
    process = subprocess.Popen(" ".join(hash_command), stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    output, error = process.communicate()
    
    if error:
        print(f"Error calculating hash: {error.decode()}")
        return None
    
    return output.decode().strip()

def compare_directories(dir1, dir2):
    differences = []

    def compare_recursive(d1, d2, relpath=""):
        dcmp = filecmp.dircmp(d1, d2)
        
        for name in dcmp.left_only:
            differences.append(f"Only in {d1} (Clone): {os.path.join(relpath, name)}")
        for name in dcmp.right_only:
            differences.append(f"Only in {d2} (Copy): {os.path.join(relpath, name)}")
        
        for name in dcmp.diff_files:
            path1, path2 = os.path.join(d1, name), os.path.join(d2, name)
            if os.path.islink(path1) != os.path.islink(path2):
                differences.append(f"Symbolic link mismatch: {os.path.join(relpath, name)}")
            elif os.path.islink(path1):
                if os.readlink(path1) != os.readlink(path2):
                    differences.append(f"Symbolic link target mismatch: {os.path.join(relpath, name)}")
            else:
                stat1, stat2 = os.stat(path1), os.stat(path2)
                if stat1.st_mode != stat2.st_mode:
                    differences.append(f"File mode mismatch for {os.path.join(relpath, name)}: {stat.filemode(stat1.st_mode)} vs {stat.filemode(stat2.st_mode)}")
                elif stat1.st_size != stat2.st_size:
                    differences.append(f"File size mismatch for {os.path.join(relpath, name)}: {stat1.st_size} vs {stat2.st_size}")
                else:
                    differences.append(f"Content mismatch: {os.path.join(relpath, name)}")
        
        for sub_dir in dcmp.common_dirs:
            new_relpath = os.path.join(relpath, sub_dir)
            compare_recursive(os.path.join(d1, sub_dir), os.path.join(d2, sub_dir), new_relpath)

    compare_recursive(dir1, dir2)
    return differences

def main():
    github_slug = "NemProject/nem"
    commit_hash = "489aad6060454d0b7b34a144e0b345c5a3a199f5"
    
    print("Cloning repository...")
    clone_time, clone_path = clone_repo(github_slug, commit_hash)
    print(f"Time taken to clone: {clone_time:.2f} seconds")
    
    print("\nCopying folder...")
    copy_time, copy_path = copy_folder(clone_path, commit_hash)
    print(f"Time taken to copy: {copy_time:.2f} seconds")
    
    print(f"\nCloning is {'faster' if clone_time < copy_time else 'slower'} than copying.")
    
    print("\nCalculating and comparing directory hashes (excluding hidden files and .git directory)...")
    clone_hash = get_directory_hash(clone_path)
    copy_hash = get_directory_hash(copy_path)
    
    print(f"Clone hash: {clone_hash}")
    print(f"Copy hash: {copy_hash}")
    
    if clone_hash == copy_hash:
        print("The contents of the cloned and copied repositories match completely (excluding hidden files and .git directory).")
    else:
        print("The contents of the cloned and copied repositories do not match.")
        print("\nDetailed comparison:")
        differences = compare_directories(clone_path, copy_path)
        for diff in differences:
            print(diff)

    # Clean up temporary directories
    shutil.rmtree(clone_path)
    shutil.rmtree(copy_path)

if __name__ == "__main__":
    main()